In [ ]:
from model import BiLSTM # don't delete it!!
from val_utils import *
import torch
device = torch.device('cuda')

In [2]:
input_sentence = '数学[mask]使用在不同的领'
model = torch.load("./model/LM.pt").to(device)

In [3]:
#删掉[mask]
def de_mask(input_sentence):
    input = list(input_sentence)
    for i in range(len(input)):
        if i + 2 <= len(input) and input[i] == '[' and input[i + 1] == 'm' and input[i + 2] == 'a':
            input[i] = '/'#'/'在词典中不存在，将以<unk>处理
            flag = i#标明位置
            del input[i+1:i + 6]
    return input, flag
print(de_mask(input_sentence))

(['数', '学', '/', '使', '用', '在', '不', '同', '的', '领'], 2)


In [4]:
def get_prob(input_sentence, flag):
    '''

    :param input_sentence:
    :return:
    '''
    L = len(list(input_sentence))
    input = torch.zeros([1, L], dtype=torch.int64).to(device)

    input_indices, _ = sentences_to_indices(input_sentence, w2i, L)

    for i in range(L):
        input[0][i] = input_indices[i]

    out = softmax(model(input).view(L, 7100), dim=1)

    indices = list(torch.argsort(out[flag], descending=True)[0:9].cpu().detach().numpy())

    for idx in range(len(indices)):
        key = indices[idx]
        word = i2w[key]
        prob = out[flag][key].cpu().detach().numpy()
        print(prob, word)

In [5]:
s, flag = de_mask(input_sentence)
get_prob(s, flag)

0.30048275 垆
0.11872694 寅
0.09722553 幂
0.08734936 阀
0.062485695 晤
0.04865133 瑙
0.028242858 敛
0.026149113 祗
0.0226325 昭
